# Regression with an Abalone Dataset
Run after following notebooks are run:
1. **01_Data_Cleaning**
2. **02_EDA**

## Development Notes/Ideas
- rf
- xgb
- knn
- lasso
- stack


## Libraries

In [1]:
## set up libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

## Load Data

In [2]:
train = pd.read_pickle('train_clean.pkl')
test = pd.read_pickle('test_clean.pkl')

## separate in to features and response variable
x_train = train.drop('Rings', axis=1)
y_train = train['Rings']

x_test = test

## One Hot Encoding

In [6]:
x_train_onehot = pd.get_dummies(x_train, drop_first=True)
x_test_onehot = pd.get_dummies(x_test, drop_first=True)
print("train dimensions: ", x_train_onehot.shape, "\n","test dimensions: ", x_test_onehot.shape)
x_train_onehot.columns.to_list()

train dimensions:  (90615, 11) 
 test dimensions:  (90615, 11)


['Length',
 'Diameter',
 'Height',
 'Whole weight',
 'Whole weight.1',
 'Whole weight.2',
 'Shell weight',
 'Volume_cube',
 'Density_meat',
 'Sex_I',
 'Sex_M']